In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 1. Cargar y limitar el dataset
# Se asume que el CSV "processedstopword1.csv" tiene al menos dos columnas: 'text' y 'label'
df = pd.read_csv("processedstopword1.csv")

# Limitar el dataset a un subconjunto, por ejemplo, 1200 muestras en total para acelerar el entrenamiento
df = df.sample(n=1200, random_state=42)

# Dividir en entrenamiento y prueba (80%/20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 2. Convertir los DataFrames a un dataset de Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 3. Seleccionar el modelo y cargar el tokenizador
# Si las noticias están en español, se puede usar "bert-base-spanish-wwm-cased"
model_name = "bert-base-spanish-wwm-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# 4. Tokenizar los textos
# Limitar la longitud de secuencia a 64 tokens para reducir la carga computacional
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=64)

train_dataset = train_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

# 5. Configurar el modelo para clasificación binaria (noticia real vs. noticia falsa)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 6. Definir argumentos de entrenamiento adaptados a recursos limitados
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,                      # Una sola época para acelerar el proceso
    per_device_train_batch_size=4,           # Tamaño de batch reducido
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",             # Evaluar al final de cada época
    save_strategy="epoch",                   # Guardar el modelo al final de cada época
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,                        # Registrar información cada 10 pasos
)

# 7. Función para calcular métricas, por ejemplo, la exactitud
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

# 8. Configurar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# 9. Entrenar el modelo
trainer.train()


Explicación de las Adaptaciones:


Limitación del Dataset:


Se reduce el número total de muestras (en este ejemplo, 1200) para disminuir el tiempo de entrenamiento.
Longitud de Secuencia:

Se fija max_length=64 en la tokenización para reducir el tamaño de entrada y, por tanto, la cantidad de cálculos necesarios.


Tamaño del Batch y Número de Épocas:


Se utiliza un tamaño de batch pequeño (per_device_train_batch_size=4) y se entrena durante una única época (num_train_epochs=1), lo que ayuda a que el entrenamiento sea viable en un entorno sin GPU.
Estas modificaciones te permitirán experimentar con el modelo en Visual Studio utilizando tu MacBook Air, y posteriormente, si deseas mejorar el desempeño o trabajar con un conjunto de datos más grande, podrías recurrir a recursos en la nube con GPU.